<a href="https://colab.research.google.com/github/kimhyeongmin-khu/MLMovieProj/blob/main/naver_movice_rate_crowling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('empty_movie_up.csv')

options = webdriver.ChromeOptions()
options.add_argument("chrome-win64/chromedriver.exe")
driver = webdriver.Chrome(options=options)

all_reviews_df=[]

for index, row in df.iterrows():
        movie_title = row['Movie_Title']
        release_date = datetime.strptime(row['Release_Date'], '%Y-%m-%d')
        url = row['URL']

        # 개봉일 전후 1,2,3주일 기간 설정

        week1_end_date = (release_date + timedelta(days=6))
        week2_end_date = (week1_end_date + timedelta(days=7))
        week3_end_date = (week2_end_date + timedelta(days=7))
        start_date = release_date



        try:
            driver.get(url)
            time.sleep(5)
            try:
                # Click on the area to load reviews
                area_element = driver.find_element(By.CLASS_NAME, 'area_card_outer')
                area_element.click()
                time.sleep(2)  # Wait for content to load (adjust as needed)

                # Find the specific element to scroll within
                scrollable_element = driver.find_element(By.CLASS_NAME, 'lego_review_list')

                # Scroll down within the specific element to load all reviews
                last_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_element)
                while True:
                    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scrollable_element)
                    time.sleep(1)  # Wait for content to load (adjust as needed)

                    new_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_element)
                    if new_height == last_height:
                        break
                    last_height = new_height

                page_source = driver.page_source


                soup = BeautifulSoup(page_source, 'html.parser')

                review_items = soup.find_all('li', class_='area_card _item')

                # Create a list to store review data
                reviews_data = []

                for review in review_items:
                    rating_id = review['data-rating-id']
                    movie_code = review['data-movie-code']
                    rating_point_type = review['data-rating-point-type']
                    writer_id = review['data-report-writer-id']
                    report_title = review['data-report-title']
                    report_time = review['data-report-time']

                    # Extract the last character as the rating value
                    rating_value = review.find('div', class_='area_text_box').text.strip()[-1]

                    if rating_value =='0':
                        rating_value='10'
                    report_time=report_time[:8]
                    report_time= datetime.strptime(report_time, '%Y%m%d')


                    reviews_data.append({
                        "Rating ID": rating_id,
                        "Movie Code": movie_code,
                        "Rating Point Type": rating_point_type,
                        "Writer ID": writer_id,
                        "Report Title": report_title,
                        "Report Time": report_time,
                        "Rating": rating_value
                    })

                    # Create a DataFrame from the reviews data
                reviews_df = pd.DataFrame(reviews_data)

                # Calculate average ratings for each week
                week1_avg = reviews_df[(reviews_df['Report Time'] >= start_date) & (reviews_df['Report Time'] <= week1_end_date)]['Rating'].astype(float).mean()
                week2_avg = reviews_df[(reviews_df['Report Time'] >= start_date) & (reviews_df['Report Time'] <= week2_end_date)]['Rating'].astype(float).mean()
                week3_avg = reviews_df[(reviews_df['Report Time'] >= start_date) & (reviews_df['Report Time'] <= week3_end_date)]['Rating'].astype(float).mean()

                # Append the results to the all_reviews_df
                all_reviews_df.append({
                    'Movie_Title': movie_title,
                    'Week1_Avg': week1_avg,
                    'Week2_Avg': week2_avg,
                    'Week3_Avg': week3_avg
                })
            except:
                all_reviews_df.append({
                    'Movie_Title': movie_title,
                    'Week1_Avg': 0,
                    'Week2_Avg': 0,
                    'Week3_Avg': 0
                })
        except Exception as e:
            print("An unexpected error occurred:", e)


driver.quit()
all_reviews_df_good = pd.DataFrame(all_reviews_df)
# Save the DataFrame to an Excel file
excel_filename = "empty_movie_up_final0608.csv"
all_reviews_df_good.to_csv(excel_filename, index=False)

print(f"Reviews with ratings saved to {excel_filename}")